In [1]:
from datasets import load_from_disk
import pandas as pd 


/Users/yiqinhuang/anaconda3/envs/nlp_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
! mkdir ../data 
! wget https://storage.googleapis.com/usc-data/all-coref-resolved.tar.gz
! wget https://storage.googleapis.com/usc-data/full-source-scored-data.jsonl.gz
! mv all-coref-resolved.tar.gz ../data
! mv full-source-scored-data.jsonl.gz ../data
! tar -xvzf ../data/all-coref-resolved.tar.gz

In [2]:
import pandas as pd
import numpy as np 
from tqdm.auto import tqdm
data_dir = '../data'
# data_dir = '../../../bloomberg-research/press-releases/data/s_p_500_backlinks'
source_df = pd.read_json(f'{data_dir}/full-source-scored-data.jsonl.gz', lines=True, compression='gzip', nrows=100)

In [3]:
article_d = load_from_disk('../all-coref-resolved')

In [4]:
filtered_article_d = article_d.filter(lambda x: x['article_url'] in set(source_df['article_url']), num_proc=10)

In [14]:
print(type(filtered_article_d))
articles = filtered_article_d.to_pandas()
print(type(articles.iloc[5]['article_text']))
for i in range(len(filtered_article_d)):
    one_article = filtered_article_d[i]
    print(i, one_article['article_url'])


<class 'datasets.arrow_dataset.Dataset'>
<class 'str'>
0 www.bbc.co.uk/blogs/researchanddevelopment/2012/10/digital-hd-radio-camera-halfrf.shtml
1 www.prweb.com/releases/2015/02/prweb12528912.htm
2 www.usatoday.com/story/news/health/2015/08/06/cvs-drops-viagra/31227269/
3 thehill.com/changing-america/well-being/mental-health/3632993-teens-confide-in-these-adults-more-than-parents-about-mental-health/
4 www.alternet.org/author/danielle-ivory
5 www.vox.com/recode/2022/9/8/23342682/why-cvs-is-spending-8-billion-to-bring-back-physician-house-calls
6 www.washingtonpost.com/wp-dyn/content/article/2009/10/15/AR2009101503137.html
7 projects.propublica.org/d4d-archive/payments/11307483
8 www.cbsnews.com/news/walgreens-minimum-wage-15-dollars-hour-october/
9 www.cnbc.com/2021/08/04/cvs-health-cvs-earnings-q2-2021.html
10 www.nj.com/news/2021/01/shoprite-to-close-dozens-of-its-pharmacies-shift-customers-to-cvs.html
11 patch.com/maryland/rockville/restaurant-inspections-exchange-deli-cheeburger-ch

In [30]:
disallowed_quote_types = set(['Other', 'Background/Narrative', 'No Quote'])

sentences_with_quotes = (
    filtered_article_d
         .to_pandas()
         .merge(source_df, on='article_url')
         [['article_url', 'attributions', 'quote_type', 'sent_lists',]]
         .explode(['attributions', 'quote_type', 'sent_lists'])
)

sentences_with_quotes = (sentences_with_quotes
     .assign(attributions=lambda df: 
             df.apply(lambda x: x['attributions'] if x['quote_type'] not in disallowed_quote_types else np.nan, axis=1)
    )
)

In [31]:
one_article = (
    sentences_with_quotes
         .loc[lambda df: df['article_url'] == df['article_url'].unique()[1]]
        .reset_index(drop=True)
)

In [26]:
doc_str = one_article[['sent_lists', 'attributions']].to_csv(sep='\t', index=False)
json_str = one_article[['sent_lists', 'attributions']].to_json(lines=True, orient='records')

I ran llama3 locally using a cool program called Ollama: https://ollama.com/. You can follow the link to install the program.

This can be useful for trying things out before we work out compute access.

Run this in your terminal:

`ollama run llama3`

Even llama3 8b is pretty powerful. Llama 70b is better, but that may not run on your local computer.

In [ ]:
import requests
import pyperclip

In [ ]:
pyperclip.copy(doc_str)
pyperclip.copy(json_str)

In [ ]:
r = requests.post(
    'http://localhost:11434/api/generate', 
    json = {
        "model": "llama3",
        "prompt":f"""
            Here is a news article, with each sentence annotated according to the source of it's information:
            ```
            {json_str}
            ```

            Please summarize each of our source annotations. Tell me in one paragraph per source: (1) who the source is (2) what informational content they provide to the article. 
            Only rely on the annotations I have provided, don't identify additional sources.
        """,
        "stream": False
})

In [ ]:
print(r.json()['response'])

In [16]:
import json
with open('../article_sum__0_100.json', 'r') as json_file:
    # Load the JSON data into a dictionary
    data = json.load(json_file)